In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS

## Preprocessing 
- API 키 불러오기 
- 데이터 불러오기 
- 데이터 전처리
    - 청크 단위 스플릿

In [2]:
# Read txt data 
loader = TextLoader("../data/data.txt",encoding="utf-8")
text=loader.load()

# Read API Key 
with open("../openai_api.txt", "r") as f: 
    api_key = f.readline()
    f.close()

# Text preprocessing -> split and make chunk
text_splitter = CharacterTextSplitter(chunk_size=30, 
                                      separator="\n\n",
                                      chunk_overlap=10)
docs = text_splitter.split_documents(text)

Created a chunk of size 67, which is longer than the specified 30
Created a chunk of size 142, which is longer than the specified 30
Created a chunk of size 163, which is longer than the specified 30
Created a chunk of size 258, which is longer than the specified 30
Created a chunk of size 76, which is longer than the specified 30
Created a chunk of size 259, which is longer than the specified 30
Created a chunk of size 313, which is longer than the specified 30
Created a chunk of size 154, which is longer than the specified 30
Created a chunk of size 35, which is longer than the specified 30
Created a chunk of size 47, which is longer than the specified 30
Created a chunk of size 35, which is longer than the specified 30
Created a chunk of size 43, which is longer than the specified 30
Created a chunk of size 46, which is longer than the specified 30
Created a chunk of size 41, which is longer than the specified 30
Created a chunk of size 73, which is longer than the specified 30
Crea

## Embedding 
- 텍스트를 넣어 임베딩 

In [ ]:
# Make Embedding using OpenAI Embedding
embeddings = OpenAIEmbeddings(openai_api_key=api_key)
# embeddings = HuggingFaceEmbeddings()

# Make embdding and change db form
db = FAISS.from_documents(docs, embeddings)

In [ ]:
print(db.index.ntotal)

# 실험 1. DB 검색 시간 이슈 
- 지속적으로 쿼리를 보내니 3번쨰 쿼리 이후로는 대답 속도가 느려지는 문제가 발생한다. 
    - DB에서 검색을 하는데 과부하가 걸리는 것으로 판단됨 

- 실험: 쿼리 사이에 랜덤한 지연 시간을 주고 쿼리를 보냈을 때 바로 응답이 나오는지 확인 

In [ ]:
queries = ["유퀴즈하고 아쉬웠던 점은 무엇인가요?", 
           "유퀴즈 할때 아쉬운점",
           "유퀴즈 찍을때 아쉬웠던 점",
           "유퀴즈하고 아쉬웠던 점", 
           "유퀴즈 하고 아쉬운점",  # 4 
           "사모님도 다시 태어나면 침착맨님과 결혼하실까요?",
           "사모님과 재결혼 의사는?",
           "사모님이 좋아요?",
           "강연 안하시는 이유가 있나요?",    # 8
           "강연 왜 안해요?",
           "말년아 우리 학교에서 수업 좀 해줘"]

In [ ]:
# 시간 지연을 적용한 코드 
import time
import random 

for query in queries:
    # random time between 5, 10
    delayed_time = random.randint(5, 10)

    # time measure start 
    start_time = time.time()

    # search 
    docs = db.similarity_search(query)

    # time measure end 
    end_time = time.time()

    # print
    print(f"Query: {query}")
    print(f"Matched doc: {docs[0].page_content}")
    print(f"Time: {end_time - start_time}")
    print("\n")
    print("----------------------------------")

    time.sleep(delayed_time)

- 처음에는 일정하게 3초-6초 정도의 응답 시간을 보이다가 약 4번째 쿼리 이후 급격히 시간이 향상하는 것을 확인하였음 
- 중간중간 다시 3초대로 응답을 하는 경우가 있음 

In [ ]:
# 시간 지연을 적용하지 않은 코드 
import time
import random 

for query in queries:
    # time measure start 
    start_time = time.time()

    # search 
    docs = db.similarity_search(query)

    # time measure end 
    end_time = time.time()

    # print
    print(f"Query: {query}")
    print(f"Matched doc: {docs[0].page_content}")
    print(f"Time: {end_time - start_time}")
    print("\n")
    print("----------------------------------")

In [ ]:
# 시간 지연을 15-20초 적용한 코드 
import time
import random 

for query in queries:
    # random time between 15, 20
    delayed_time = random.randint(15, 20)

    # time measure start 
    start_time = time.time()

    # search 
    docs = db.similarity_search(query)

    # time measure end 
    end_time = time.time()

    # print
    print(f"Query: {query}")
    print(f"Matched doc: {docs[0].page_content}")
    print(f"Time: {end_time - start_time}")
    print("\n")
    print("----------------------------------")

    time.sleep(delayed_time)

- 결과: 15, 20초 가량의 지연 시간을 제공하니 즉각적인 응답을 제공하는 것을 확인하였음 
    - 하나의 쿼리를 처리하는데 제한되는 시간이 있는 것으로 판단됨 

## 1-1. Chorma DB로 변경 
- FAISS에서는 검색이 누적되면은 검색 시간이 증가하는 문제가 발생하였음 
- 이에 검색 데이터베이스를 Chorma로 변경하여 검색 시간에 이상이 없는지 확인하고자 함 

In [ ]:
from langchain_community.vectorstores import Chroma

# Make Embedding using OpenAI Embedding
embeddings = OpenAIEmbeddings(openai_api_key=api_key)
# embeddings = HuggingFaceEmbeddings()

# Make embdding and change db form
db = Chroma.from_documents(docs, embeddings)

In [ ]:
queries = ["유퀴즈하고 아쉬웠던 점은 무엇인가요?", 
           "유퀴즈 할때 아쉬운점",
           "유퀴즈 찍을때 아쉬웠던 점",
           "유퀴즈하고 아쉬웠던 점", 
           "유퀴즈 하고 아쉬운점",  # 4 
           "사모님도 다시 태어나면 침착맨님과 결혼하실까요?",
           "사모님과 재결혼 의사는?",
           "사모님이 좋아요?",
           "강연 안하시는 이유가 있나요?",    # 8
           "강연 왜 안해요?",
           "말년아 우리 학교에서 수업 좀 해줘"]

In [ ]:
# 시간 지연을 적용하지 않은 코드 
import time

for query in queries:
    # time measure start 
    start_time = time.time()

    # search 
    docs = db.similarity_search(query)

    # time measure end 
    end_time = time.time()

    # print
    print(f"Query: {query}")
    print(f"Matched doc: {docs[0].page_content}")
    print(f"Time: {end_time - start_time}")
    print("\n")
    print("----------------------------------")

- 결과: Chroma DB도 동일한 문제가 발생하는 것을 확인하였음 
    - 이건 langchain 자체의 문제라고 판단됨 